In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt

pd.set_option("display.max_colwidth", None)  # This will be helpful when visualizing retriever outputs

In [ ]:
cd RAG-docs/processed/

c:\Users\wat6sv\OneDrive - University of Virginia\SDS_DataScientist\Student Research\IA-in-HigherEd\RAG-docs\processed


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load example document
with open("Week7-lecture.txt") as f:
    state_of_the_union = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    # is_separator_regex=True,
    # separators="\n"
)
texts = text_splitter.create_documents([state_of_the_union])
display(texts[0], texts[1])

Document(page_content="okay hello welcome to uh week 11's lecture uh it's been a little while since we")

Document(page_content="talked about some of these things we had the kind of uh with the package presentations in the middle but we're going to talk focus\nmostly on supervised machine learning we're gonna get into uh really")